In [1]:
import tensorflow as tf
import numpy as np
import os
import math

/Users/praveen/Library/Python/3.7/lib/python/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/praveen/Library/Python/3.7/lib/python/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/praveen/Library/Python/3.7/lib/python/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/praveen/Library/Python/3.7/lib/python/site-packages

In [2]:
IMG_WIDTH = 288
IMG_HEIGHT = 288
learning_rate = 50
batch_size = 4


In [3]:
def get_label(file_path):
    return tf.strings.regex_full_match(file_path, ".*dog\..*")

def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    return img

def process_path(file_path):
    print(f"It's a {file_path}")
    return decode_img(tf.io.read_file(file_path)), get_label(file_path)

In [4]:
train_dir = "/Users/Praveen/Desktop/data/dogs-vs-cats/train/"
dataset = tf.data.Dataset.list_files(os.path.join(train_dir, "*.*.jpg"))
dataset = dataset.map(process_path)

dataset.shuffle(buffer_size=100)
data = dataset.batch(4)

iterator = data.make_initializable_iterator()
batch = iterator.get_next()

It's a Tensor("args_0:0", shape=(), dtype=string)
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


In [5]:
def inference(images):
    conv1 = tf.layers.conv2d(images, 32, 5, activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(conv1, 2, 2)
    conv2 = tf.layers.conv2d(pool1, 64, 4, activation=tf.nn.relu)
    flat = tf.layers.flatten(conv2)
    dense = tf.layers.dense(flat, 2014)
    output_layer = tf.layers.dense(dense, 2)
    return output_layer

In [6]:
def losses(logits, labels):
    return tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)

def training(loss, learning_rate):
    return tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

def evaluation(logits, labels):
    return tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits, labels, 1), tf.float32))

In [7]:
x = tf.placeholder(tf.float32, shape=[None, IMG_WIDTH, IMG_HEIGHT, 3])
y = tf.placeholder(tf.int32, shape=[None])

model = inference(x)
loss = losses(model, y)
acc = evaluation(model, y)
train_op = training(loss, learning_rate)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


In [ ]:
epochs = 10
print("A")
with tf.Session() as sess:
    print("B")
    sess.run(tf.global_variables_initializer())
    print("C")
    sess.run(iterator.initializer)
    print("D")
    for epoch in range(epochs):
        print(f"Welcome to {epoch}")
        step = 0
        while True:
            try:
                print(f"Welcome to {step}")
                images, labels = sess.run(batch)
                l, accuracy, _ = sess.run([loss, acc, train_op], feed_dict={x: images, y: labels})
                step += 1
                if step % 1 == 0:
                    print(f"{l}, {accuracy}")
            except tf.errors.OutOfRangeError:
                break

A
B
C
D
Welcome to 0
Welcome to 0
[0.86647713 0.9234288  0.8660153  0.4945441 ], 0.25
Welcome to 1
[2.5617161e+11 0.0000000e+00 0.0000000e+00 3.0236787e+11], 0.5
Welcome to 2
[7.075114e+10 0.000000e+00 0.000000e+00 0.000000e+00], 0.75
Welcome to 3
